### Dash Library 
Dash is a python framework created by ``plotly`` for creating interactive web applications. Dash is written on the top of Flask, Plotly.js and React.js.  Dash is open source and the application build using this framework are viewed on the web browser.

### Components of Dash Apps

Dash applications are made up of 2 components :
1. Layout
2. Callbacks

### 1. Layout
    
Layout describes the look and feel of the app, it defines the elements such as graphs, dropdowns etc and the placement, size, color etc of these elements. Dash contains:
#### a. Dash HTML components 
The dash_html_components (dash.html as of Dash v2.0) function has a component for every HTML tag. The ```html.H1(children='Hello Dash')``` component generates a ```<h1>Hello Dash</h1>``` HTML element in your application.

#### b.Dash Core Components
Not all components are pure HTML. The dash_core_components describe higher-level components that are interactive and are generated with JavaScript, HTML, and CSS through the React.js library.

### 2. Callbacks
Callbacks are the functions that are automatically called by Dash whenever an input component's property changes, in order to update some property in another component (the output).


### Installation:
Install the following libraries:

``pip install dash``   
``pip install plotly``

### Corona Virus Dataset visualization using Dash

After installation, we will write the following code in a python script ``coronaapp.py``

```
import dash
from dash import html
from dash import dcc
import plotly.express as px
import pandas as pd
import numpy as np

app = dash.Dash()   #initialising dash app
dataset = pd.read_csv('corona dataset Pakistan/PK COVID-19-3jun.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
datewise_add = dataset.pivot_table(index=['Date'],aggfunc=np.sum)

def recovered_cases():
	fig = px.line(datewise_add, x=datewise_add.index, y=datewise_add['Recovered'], title="Trend analysis") 
	return fig

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Covid 19 Analysis', style = {'textAlign':'center'}),
    dcc.Graph(id = 'line_plot', figure = recovered_cases())    
    ])
	
if __name__ == '__main__': 
    app.run_server()

```
In the above code, we first import the required libraries. Then we intialized Dash app in a similar way as done for Flask app. Next, we read the dataset and create a pivot table to summarize the counts of deaths and recovered cases in different cities of Pakistan.

``app.layout`` is then initialized using html Div component, which is a kind of wrapper within which the elements(heading, graph) of the layout will be created. The Div component contains arguments such as id (a unique identifier of the element), style (for setting the width, height, color etc) and children (equal to square bracket within which elements of the layout are initialised).

Inside ``html.Div``, we are using dash core component (dcc) to create graph , where we are setting the id of the graph and the figure argument, which is equal to the function call (recovered_cases()) that returns the plotly figure object.

### Using Dash Core Components

#### DropDown:

Now we will create a Dropdown to switch between Death Cases and Recovered cases. 

```
dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'Recovered Cases', 'value':'Recovered' },
            {'label': 'Death Cases', 'value':'Deaths'}
            
            ],
        value = 'Recovered')
```
Dropdowns are created using the ```Dropdown()``` function, which has the following arguments -

1. id — Unique identifier of the dropdown.
2. options — Sets the ‘label’ (the text visible in the dropdown) and ‘value’ (used by dash to communicate with callbacks) as key value pair.
3. value — default selection for the dropdown.

Now, lets have a look at how we can create the callback that will connect the dropdown and trend analysis line chart.

A callback is initialised using @app.callback(), which is followed by a function definition. Within this function, we define what happens on changing the value of the dropdown.

```
import dash
from dash import html
from dash import dcc
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output

app = dash.Dash(__name__)   #initialising dash app
dataset = pd.read_csv('corona dataset Pakistan/PK COVID-19-3jun.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
datewise_add = dataset.pivot_table(index=['Date'],aggfunc=np.sum)


app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Covid 19 Analysis', style = {'textAlign':'center'}),
    dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'Recovered Cases', 'value':'Recovered' },
            {'label': 'Death Cases', 'value':'Deaths'}
            
            ],
        value = 'Recovered'),
        dcc.Graph(id = 'line_plot')
	])
@app.callback(Output(component_id='line_plot', component_property= 'figure'),
              Input(component_id='dropdown', component_property= 'value'))
def graph_update(dropdown_value):
	print(dropdown_value)
	fig = px.line(datewise_add, x=datewise_add.index, y=datewise_add[dropdown_value], title="Trend analysis") 
	return fig  

	
if __name__ == '__main__': 
    app.run_server()

```
Lets have a look at the arguments within the callback function :

**1. Output:**
This is used to define the components within the layout which will be updated when the function below the callback (graph_update()) return some object. Output function takes 2 arguments 
   1. component_id: defines the id of the component we want to update with our function graph_update. We want to update trend analysis chart within the dcc.Graph, so we will set the component id to ‘line_plot’, which is id of our graph component. 
   2. Component property: defines the property of the component that will be updated, which is figure property of dcc.Graph in our layout.
   
**2. Input:**
This is used to define the components, the change in whose value will trigger the callback. Input function also takes component_id and component_property as argument. We want the callback to get triggered based on change in the value of our dropdown, so we set the component_property to ‘value’ property of dropdown. Please note that Input is defined within a list.


### File Uploading Using Dash

```
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import base64
import os
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Upload(id='upload-data',children=html.Button('Upload File')),
    html.Hr(),
	html.Div(id='output-data-upload')
])
def save_file(name, content):
	"""Decode and store a file uploaded with Plotly Dash."""
	data = content.encode("utf8").split(b";base64,")[1]
	with open(name, "wb") as fp:
		fp.write(base64.decodebytes(data))
	f = open(name,"r")
	return f.read()


@app.callback(Output('output-data-upload', 'children'),
              Input("upload-data", "filename"),
			  Input("upload-data", "contents")
             )
def read_output(fname,content):
	print(fname)
	print(content)
	return save_file(fname,content)
	
        


if __name__ == '__main__':
    app.run_server()

```

### Multi Page App

For multipage applications, read documentation: https://dash.plotly.com/urls